<img src="images/DBG1.jpg">
Photo credit: Caitlin Buck

# Preparation of saguaro data for mortality analysis

### Import Saguaro Inventory dataset  obtained from Desert Botanical Garden
* Download requested from https://livingcollections.org/dbg/Home.aspx on 10/18/20
* Full history of inventory and measurement data for all saguaros alive and dead.csv

### Clean up and reformat data
* Drop columns not required for analysis.
* Parse year from Inventory Date column for use in subsequent clean-up.
* Select inventories where saguaros were marked as dead ("forever dormant").
* Find saguaros that had more than one inventory in a given year, and remove the extra inventories.
* Remove inventories reported prior to timespans of interest.
* Exclude saguaro deaths unrelated to weather analysis.

### Derive data needed for subsequent analysis
* Count the number of saguaros that died in each year.
* Group inventories into the two timespans of interest.
* Count the number of saguaros that died in each timespan.

### Export datasets to csv
* Mortality_per_year_ready_for_analysis.csv
* Mortality_per_timespan_ready_for_analysis.csv

<img src="images/Dirty data.png">

### Import dependencies, read in dataset and store in pandas DataFrame.

In [1]:
# Import dependencies
import pandas as pd
import numpy as py

In [2]:
# Read in saguaro dataset with cp1252 character encoding and store in pandas DataFrame called full_df.
full_data = "../Original datafiles/Full history of inventory and measurement data for all saguaros alive and dead.csv"
full_df = pd.read_csv(full_data, encoding='cp1252')

full_df

,Accession Planting,Inventory Date,Status,Condition,Measurement Date,Height,Height Unit,DBH,DBH Unit,Measurement Note
0,2020-0589-10-0,9/17/2020 0:00,Dormant,Good,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-0589-10-0,9/18/2020 17:27,Dormant,Good,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-0588-10-0,9/17/2020 0:00,Dormant,Good,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-0588-10-0,9/18/2020 17:27,Dormant,Good,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-0587-10-0,9/17/2020 0:00,Dormant,Good,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
13850,1947-1548-01-1,6/17/2009 0:00,Alive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13851,1947-1548-01-1,7/9/2012 0:00,Alive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13852,1947-1548-01-1,11/28/2012 12:16,Alive,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13853,1947-1548-01-1,12/17/2012 0:00,Dead,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Drop columns not required for analysis.

In [3]:
# From full_df, drop columns not needed for mortality analysis, leaving only the inventory event columns.
# Drop NaN rows (representing Measurement events rather than Alive/Dead Inventory events.)
# Save the remaining (inventory) columns as new DataFrame called inventory_df.
inventory_df = full_df.drop(["Condition", "Measurement Date", "Height", "Height Unit", "DBH", "DBH Unit", "Measurement Note"], axis = 1)
drop_df = inventory_df.dropna()
drop_df

,Accession Planting,Inventory Date,Status
0,2020-0589-10-0,9/17/2020 0:00,Dormant
1,2020-0589-10-0,9/18/2020 17:27,Dormant
2,2020-0588-10-0,9/17/2020 0:00,Dormant
3,2020-0588-10-0,9/18/2020 17:27,Dormant
4,2020-0587-10-0,9/17/2020 0:00,Dormant
...,...,...,...
13850,1947-1548-01-1,6/17/2009 0:00,Alive
13851,1947-1548-01-1,7/9/2012 0:00,Alive
13852,1947-1548-01-1,11/28/2012 12:16,Alive
13853,1947-1548-01-1,12/17/2012 0:00,Dead


### Parse year from Inventory Date column for use in subsequent clean-up.

In [4]:
# Extract year from Inventory Date column and store in new column called Inventory Year. Drop Inventory Date column as
# it's no longer needed.
inventory_year = drop_df["Inventory Date"].str.extract(r'(20\d\d)')
drop_df["Inventory Year"] = inventory_year
inventory_drop_df = drop_df.drop(["Inventory Date"], axis = 1)

inventory_drop_df

<ipython-input-4-499ca5eaf16f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_df["Inventory Year"] = inventory_year


,Accession Planting,Status,Inventory Year
0,2020-0589-10-0,Dormant,2020
1,2020-0589-10-0,Dormant,2020
2,2020-0588-10-0,Dormant,2020
3,2020-0588-10-0,Dormant,2020
4,2020-0587-10-0,Dormant,2020
...,...,...,...
13850,1947-1548-01-1,Alive,2009
13851,1947-1548-01-1,Alive,2012
13852,1947-1548-01-1,Alive,2012
13853,1947-1548-01-1,Dead,2012


### Select inventories where saguaros were marked as dead ("forever dormant").

In [5]:
# From inventory_df, select inventories where a saguaro was marked as dead ("forever dormant)".
# Save the selection as new DataFrame called forever_dormant_df.
forever_dormant_df = inventory_drop_df.loc[full_df["Status"] == "Dead"]

forever_dormant_df

,Accession Planting,Status,Inventory Year
73,2019-0188-01-12,Dead,2019
80,2019-0142-01-2,Dead,2020
403,2018-0026-01-5,Dead,2019
740,2016-0042-01-4,Dead,2018
741,2016-0042-01-4,Dead,2018
...,...,...,...
13830,1966-8295-01-2,Dead,2016
13847,1949-2064-01-1,Dead,NaN
13848,1947-1600-01-1,Dead,NaN
13853,1947-1548-01-1,Dead,2012


### Find saguaros that had more than one inventory and remove the extra inventories.

In [6]:
# View DataFrame row count before removing duplicate inventories
rows_before = len(forever_dormant_df)

print(f"Before removing duplicates, there are {rows_before} saguaro loss inventories present.")

Before removing duplicates, there are 323 saguaro loss inventories present.


In [7]:
# Sort DataFrame by year so earliest inventory will be kept when duplicates are dropped. (Assuming that the first
# time a saguaro was inventoried as dead is the closest date to its actual death. Later inventories were likely
# saved to add cause of death information to the record.)
sorted_df = forever_dormant_df.sort_values(by = "Inventory Year")

# Remove duplicate accessions (but keep first duplicate)
unique_df = sorted_df.drop_duplicates(subset="Accession Planting", keep="first", inplace = False)

# View DataFrame row count after removing duplicates
rows_after = len(unique_df)
duplicates_removed = rows_before - rows_after

print(f"After removing {duplicates_removed} duplicates, there are {len(unique_df)} inventories kept for analysis.")

After removing 48 duplicates, there are 275 inventories kept for analysis.


### Remove inventories reported prior to timespans of interest.

In [8]:
# Remove inventories prior to 2014; keep 2014 and on.
# used in next step.
twentythirteen_and_on_df = unique_df.loc[pd.to_numeric(unique_df["Inventory Year"]) >= 2014]

rows_before = len(unique_df)
rows_after = len(twentythirteen_and_on_df)
rows_removed = rows_before - rows_after

print(f"After removing {rows_removed} rows, there are {rows_after} inventories kept for analysis.")

After removing 118 rows, there are 157 inventories kept for analysis.


In [9]:
# View current DataFrame
twentythirteen_and_on_df

,Accession Planting,Status,Inventory Year
1171,2014-0302-01-1,Dead,2014
6614,2012-0254-01-9,Dead,2014
8376,2012-0065-01-50,Dead,2014
8934,2008-0149-01-1,Dead,2014
11182,1987-0156-01-7,Dead,2014
...,...,...,...
12145,1985-0254-01-24,Dead,2020
8845,2012-0065-01-14,Dead,2020
10885,1990-0214-01-6,Dead,2020
3422,2012-0315-01-36,Dead,2020


### Exclude saguaro deaths unrelated to weather analysis

#### Many saguaros were inventoried as dead for reasons unrelated to weather:
* Saguaros that died in the propagation facilities
* Saguaros that were inventoried as dead many years after the event of their death
* Saguaro IDs that were deactivated because they were never assigned to an actual plant
* Saguaro IDs that were mis-recorded as Dead at some point in the past, but have been corrected to Alive. E.g., a juvenile saguaro that was well-hidden under a shrub one year.

#### A list of inventories unrelated to the weather analysis was obtained from a project member familiar with the inventory protocol. The list is imported, joined to the DataFrame, and used to select inventories appropriate for the weather analysis. The list is exported to csv for subsequent analysis.

In [26]:
# Get row count before dropping rows.
rows_before = len(twentythirteen_and_on_df)

# Import exclusion list and merge it with twentythirteen_and_on_df using a full outer join. Store information about origin of
# each row for exclusion step.
exclude_df = pd.read_csv("../Original datafiles/Saguaros to exclude.csv", encoding = "cp1252")
merged_df = pd.merge(twentythirteen_and_on_df, exclude_df, on = "Accession Planting", how='outer', indicator=True)
after_exclusion_df = merged_df.loc[merged_df["_merge"] == "left_only"]

after_exclusion_df.to_csv("../Data ready for analyses/after_exclusion_df.csv", encoding = 'utf-8')

# Review effect of dropping rows.
rows_after = len(after_exclusion_df)
rows_removed = rows_before - rows_after
print(f"After removing {rows_removed} saguaro losses not related to weather, there are {rows_after} inventories kept for weather analysis.")

after_exclusion_df.head()

After removing 58 saguaro losses not related to weather, there are 99 inventories kept for weather analysis.


,Accession Planting,Status,Inventory Year,_merge
0,2014-0302-01-1,Dead,2014,left_only
1,2012-0254-01-9,Dead,2014,left_only
2,2012-0065-01-50,Dead,2014,left_only
3,2008-0149-01-1,Dead,2014,left_only
4,1987-0156-01-7,Dead,2014,left_only


### Count the number of saguaros that died in each year and export to csv for subsequent analysis

In [27]:
# Aggregate saguaro mortality to counts for each year of interest and reset table index.
mortality_per_year_agg = after_exclusion_df.groupby(["Inventory Year"])["Accession Planting"].agg("count").reset_index()

# Convert aggregation object to DataFrame.
mortality_per_year_df = pd.DataFrame(mortality_per_year_agg)

mortality_per_year_df

,Inventory Year,Accession Planting
0,2014,5
1,2015,21
2,2016,8
3,2017,19
4,2018,13
5,2019,18
6,2020,15


In [28]:
# Rename columns to represent the data they now contain.
mortality_per_year_df.rename(columns = {"Inventory Year": "Year", "Accession Planting": "Mortality Count"}, inplace = True)

# Export to csv.
mortality_per_year_df.to_csv("../Data ready for analyses/Mortality_per_year_ready_for_analysis.csv", encoding = 'utf-8')

mortality_per_year_df

,Year,Mortality Count
0,2014,5
1,2015,21
2,2016,8
3,2017,19
4,2018,13
5,2019,18
6,2020,15


### End of mortality rate data prep